In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set GPU device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Set the path to your tea leaf dataset
train_dir = '/content/drive/MyDrive/tea_leaf/ratio3:1/train'
valid_dir = '/content/drive/MyDrive/tea_leaf/ratio3:1/val'
test_dir = '/content/drive/MyDrive/tea_leaf/ratio3:1/test'

# Set the image dimensions
img_width, img_height = 224, 224

# Set the number of classes (disease categories)
num_classes = 8

# Set the batch size and number of epochs
batch_size = 10
epochs = 30

# Create data generators with data augmentation for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512, activation='relu')(x)

# Add the final classification layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze the last few layers of the VGG16 model for fine-tuning
for layer in model.layers[-10:]:
    layer.trainable = True

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
with tf.device('/GPU:0'):
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.samples // batch_size,
        epochs=epochs
    )

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Found 706 images belonging to 8 classes.
Found 87 images belonging to 8 classes.
Epoch 1/30
70/70 [==============================] - 405s 5s/step - loss: 1.9152 - accuracy: 0.2529 - val_loss: 1.5192 - val_accuracy: 0.4750
Epoch 2/30
70/70 [==============================] - 78s 1s/step - loss: 1.1493 - accuracy: 0.5675 - val_loss: 1.1070 - val_accuracy: 0.5250
Epoch 3/30
70/70 [==============================] - 81s 1s/step - loss: 0.7616 - accuracy: 0.6710 - val_loss: 0.5705 - val_accuracy: 0.7875
Epoch 4/30
70/70 [==============================] - 79s 1s/step - loss: 0.6502 - accuracy: 0.7241 - val_loss: 0.6278 - val_accuracy: 0.7000
Epoch 5/30
70/70 [==============================] - 79s 1s/step - loss: 0.6098 - accuracy: 0.7399 - val_loss: 0.6285 - val_accuracy: 0.7125
Epoch 6/30
70/70 [==============================] - 80s 1s/step - loss: 0.5615 - accuracy: 0.7572 - val_loss: 0.5426 - val_accuracy: 0.7500
Epoch 7/30
70/70 [==============================] - 85s 1s/step - loss: 0.4961

In [7]:
# Save the trained model
model.save('/content/drive/MyDrive/tea_leaf/tea_leaf_model.h5')

In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [5]:
# Load Model
model=load_model('/content/tea_leaf_model.h5')

In [6]:
img=image.load_img(r'/content/drive/MyDrive/tea_leaf/train/Anthracnose/IMG_20220503_143344.jpg',target_size=(224,224))
#convert image to array format
x=image.img_to_array(img)
import numpy as np
x=np.expand_dims(x,axis=0)
img_data = preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index=['Anthracnose','algal leaf','bird eye spot','brown blight','gray light','healthy','red leaf spot','white spot']
result = str(index[output[0]])
result

1/1 [==============================] - 1s 733ms/step


'Anthracnosealgal leaf'